<a href="https://colab.research.google.com/github/Muhirwakyeyune/AMMI-NATURAL-LANGUAGE-PROCESSING/blob/main/Muhirwa_Salomon_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [39]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile

    Returns:
    data (list of lists): each list is a sentence of the text
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [40]:

print("load training set..")
print("\n")
train_data, vocab = load_data("/content/drive/MyDrive/session3/train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("/content/drive/MyDrive/session3/valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [41]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count

    Returns:
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab
    # or have a count that is below mincount
    data_with_unk = []

    ## FILL CODE
    for sentence in data:
      sentence_with_unk=[]
      for word in sentence:
        if word not in vocab or vocab[word] <= mincount:
          sentence_with_unk.append('<unk>')
        else:
          sentence_with_unk.append(word)
      data_with_unk.append(sentence_with_unk)
    return data_with_unk

In [42]:

print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0])


remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [53]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text
    n (int): size of the n-gram

    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value
        for i in range(len(sentence) -n + 1):
          context=sentence[i:i + n-1]
          word=sentence[i +  n- 1]
          counts[context][word] +=1
          total_number_words +=1


    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():
    ## FILL CODE
      context_total = sum(counts[context].values())
      for word in counts[context].keys():
          prob[context][word] = counts[context][word] / context_total

    return prob

In [54]:
# RUN TO BUILD NGRAM MODEL

n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  2


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [55]:
def get_prob(model, context, w):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word: probability of this word given context}
    }
    context (list of strings): a sentence
    w(string): the word we need to find its probability given the context

    Returns:
    prob(float): probability of this word given the context
    '''
    # code a recursive function over
    # smaller and smaller context
    # to compute the backoff model

    ## FILL CODE


    if len(context) == 0:
        return 0.0

    # Convert the context to a tuple
    context = tuple(context)

    if context in model and w in model[context]:
        return model[context][w]

    # If the context is not in the model, recursively backoff to smaller context
    return get_prob(model, context[1:], w)

In [ ]:
def perplexity(model, data, n):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram

    Retunrs:
    perp(float): the perplexity of the model
    '''
    ## FILL CODE

    return 0.0

In [60]:


def perplexity(model, data, n):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word: probability of this word given context}
    }
    data (list of lists): each list is a sentence of the text
    n (int): size of the n-gram

    Returns:
    perp (float): the perplexity of the model
    '''

    total_log_prob = 0.0
    total_words = 0

    for sentence in data:
        sentence = tuple(sentence)

        for i in range(n - 1, len(sentence)):
            context = sentence[i - n + 1:i]
            word = sentence[i]

            # Compute the log probability of the word given the context
            log_prob = np.log(get_prob(model, context, word) + 1e-10)

            total_log_prob += log_prob
            total_words += 1

    # Compute the average log probability
    avg_log_prob = total_log_prob / total_words

    # Compute perplexity as the exponentiation of the negative average log probability
    perp = math.exp(-avg_log_prob)

    return perp

In [61]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 819.468085403767


In [ ]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and
    thier probabilites

    Retunrs:
    words_and_probs(dic): {word: probability of word given context}

    '''
    # code a recursive function over context
    # to find the longest available ngram

    ## FILL CODE

    return None

In [62]:
def get_proba_distrib(model, context):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word: probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and
    their probabilities

    Returns:
    words_and_probs (dict): {word: probability of word given context}
    '''

    words_and_probs = {}

    # Start with the full context and gradually reduce it until a matching n-gram is found
    while len(context) > 0:
        if tuple(context) in model:
            words_and_probs = model[tuple(context)]
            break
        else:
            context = context[1:]

    return words_and_probs

In [ ]:
def generate(model):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }

    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model.


    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE

    return sentence

In [63]:
import numpy as np

def generate(model):
    '''
    Parameters:
    model (dictionary of dictionary)
    {
        context: {word: probability of this word given context}
    }

    Returns:
    sentence (list of strings): a sentence sampled according to the language model.
    '''

    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence) < 100:
        context = sentence[-n:]  # Get the last n words as the context
        word_probs = get_proba_distrib(model, context)  # Get the word probability distribution given the context
        words = list(word_probs.keys())
        probs = list(word_probs.values())
        next_word = np.random.choice(words, 1, p=probs)[0]  # Sample the next word based on the probabilities
        sentence.append(next_word)

    return sentence

In [64]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'as', 'it', 'was', 'traveling', 'by', 'ourselves.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.